# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298471160156                   -0.85    5.1
  2   -8.300195818505       -2.76       -1.25    1.2
  3   -8.300445893920       -3.60       -1.89    3.2
  4   -8.300462560575       -4.78       -2.76    1.6
  5   -8.300464397418       -5.74       -3.05    6.2
  6   -8.300464506250       -6.96       -3.21    1.0
  7   -8.300464571278       -7.19       -3.35    7.0
  8   -8.300464607044       -7.45       -3.48    1.1
  9   -8.300464634684       -7.56       -3.68    1.0
 10   -8.300464637068       -8.62       -3.73    2.5
 11   -8.300464641961       -8.31       -3.93    1.0
 12   -8.300464643954       -8.70       -4.43    1.1


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.64184707839                   -0.70    5.5
  2   -16.67794726043       -1.44       -1.14    1.4
  3   -16.67919566677       -2.90       -1.89    4.0
  4   -16.67924973553       -4.27       -2.68    4.2
  5   -16.67928411460       -4.46       -3.08    4.9
  6   -16.67928620822       -5.68       -3.52    1.9
  7   -16.67928622056       -7.91       -4.02    1.2


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32515078379                   -0.56    5.9
  2   -33.33244144530       -2.14       -1.01    1.0
  3   -33.33409834628       -2.78       -1.75    5.8
  4   -33.33516266168       -2.97       -2.36    3.0
  5   -33.33692425543       -2.75       -2.57   15.1
  6   -33.33694159188       -4.76       -2.98    3.4
  7   -33.33694389362       -5.64       -3.91    1.9
  8   -33.33694391123       -7.75       -4.16    5.8


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298594041736                   -0.85    5.4
  2   -8.300292989946       -2.77       -1.59    1.0
  3   -8.300462093409       -3.77       -2.70    4.4
  4   -8.300452602092   +   -5.02       -2.71    7.9
  5   -8.300464400589       -4.93       -3.57    1.5
  6   -8.300464605241       -6.69       -4.01    3.0


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32521879587                   -0.56    6.9
  2   -33.27779487292   +   -1.32       -1.25    1.0
  3   +11.87964956709   +    1.65       -0.24   11.6
  4   -33.28204427750        1.65       -1.47    7.5
  5   -33.28243502532       -3.41       -1.40    4.2
  6   -33.30958049135       -1.57       -1.80    2.9
  7   -32.72384966506   +   -0.23       -1.17    4.9
  8   -33.33086238146       -0.22       -2.08    5.2
  9   -33.33600484349       -2.29       -2.26    2.9
 10   -33.33681227413       -3.09       -2.62    2.4
 11   -33.33692543873       -3.95       -2.96    2.1
 12   -33.33693151168       -5.22       -3.04    1.6
 13   -33.33693896839       -5.13       -3.39    1.9
 14   -33.33694355545       -5.34       -3.73    2.4
 15   -33.33694349267   +   -7.20       -4.09    3.0


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.